In [9]:
import os
import nrrd
import numpy as np
from helpers import *
from ipywidgets import interact, IntSlider
import matplotlib.pyplot as plt
import shutil
from scipy.ndimage import binary_dilation, binary_erosion, binary_closing

from concurrent.futures import ProcessPoolExecutor

In [6]:
#helper to move and rename nrrd files
def rename_and_move_nrrd_files(root_dir):
    for subdir, _, files in os.walk(root_dir):
        if os.path.basename(subdir) == 'original_labels':
            raw_dir = os.path.join(os.path.dirname(subdir), 'raw')
            os.makedirs(raw_dir, exist_ok=True)
            
            for file in files:
                old_path = os.path.join(subdir, file)
                
                if file.endswith('_data.nrrd'):
                    new_filename = file.replace('_data.nrrd', '_raw.nrrd')
                    new_path = os.path.join(raw_dir, new_filename)
                elif file.endswith('_raw.nrrd'):
                    new_path = os.path.join(raw_dir, file)
                else:
                    continue  # Skip files that do not match the criteria
                
                shutil.move(old_path, new_path)
                print(f"Moved and renamed: {old_path} -> {new_path}")

In [7]:
if False:
    current_directory = os.getcwd()
    root_directory = f'{current_directory}/data/Vesuvius/'
    rename_and_move_nrrd_files(root_directory)

In [10]:
def process_nrrd_file(nrrd_path, label_dir, weight_dir, erosion_iterations=1, dilation_iterations=1):
    print(f"Processing: {nrrd_path}")
    data, header = nrrd.read(nrrd_path)
    unique_values = np.unique(data[data > 0])  # Ignore background & masking

    # Create an empty array for the result
    result = np.zeros_like(data, dtype=np.uint8)
    weights = np.ones_like(data, dtype=np.uint8)
    border_overlap = np.zeros_like(data, dtype=np.uint8)

    for value in unique_values:
        structure_mask = data == value
        
        # Pad the structure mask to prevent erosion at the edges
        padded_structure = np.pad(structure_mask, pad_width=erosion_iterations, mode='constant', constant_values=value)
        
        # Erode the padded structure
        eroded_padded_structure = binary_erosion(padded_structure, iterations=erosion_iterations)
        
        # Remove the padding after erosion
        eroded_structure = eroded_padded_structure[
            erosion_iterations:-erosion_iterations,
            erosion_iterations:-erosion_iterations,
            erosion_iterations:-erosion_iterations
        ]
        
        # Ensure the eroded structure is within bounds
        if eroded_structure.shape != structure_mask.shape:
            print(f"WARNING: Erosion caused shape mismatch for value {value}")
            eroded_structure = np.zeros_like(structure_mask)
        
        if dilation_iterations > 0:
            # Dilate the original structure
            dilated_structure = binary_dilation(structure_mask, iterations=dilation_iterations)
        else:
            dilated_structure = structure_mask
        
        border = np.logical_xor(dilated_structure, eroded_structure)
        # Label assignments
        result[border] = 1  # Border class
        result[eroded_structure] = 2  # Foreground class (overwrites the border if applicable)
        
        # Weight assignments
        weights[border] = 2 # border weight
        weights[eroded_structure] = 1

        # Update border overlap array
        border_overlap += border.astype(np.uint8)

    # Assign a weight of 10 to overlapping border regions
    weights[border_overlap > 1] = 10
    
    # Save the processed arrays back to .nrrd files
    label_output_path = os.path.join(label_dir, f"tri_class_{os.path.basename(nrrd_path)}")
    weight_output_path = os.path.join(weight_dir, f"weights_{os.path.basename(nrrd_path)}")
    nrrd.write(label_output_path, result, header)
    nrrd.write(weight_output_path, weights, header)
    print(f"Processed and saved: {label_output_path} and {weight_output_path}")

def process_nrrd_files(root_dir):
    for subdir, _, files in os.walk(root_dir):
        if os.path.basename(subdir) == 'original_labels':
            # Determine the corresponding 'label' directory
            label_dir = os.path.join(os.path.dirname(subdir), 'label')
            os.makedirs(label_dir, exist_ok=True)
            weight_dir = os.path.join(os.path.dirname(subdir), 'weight')
            os.makedirs(weight_dir, exist_ok=True)

            # Prepare the arguments for parallel processing
            nrrd_paths = [os.path.join(subdir, file) for file in files if file.endswith('.nrrd')]
            args = [(nrrd_path, label_dir, weight_dir) for nrrd_path in nrrd_paths]
            # for a in args:
            #     print(f"Queueing: {a}")

            # Use ProcessPoolExecutor for parallel processing
            with ProcessPoolExecutor() as executor:
                futures = [executor.submit(process_nrrd_file, *arg) for arg in args]
                for future in futures:
                    future.result()  # This will re-raise any exceptions caught during processing


In [11]:
import os

# Get the current working directory
current_directory = os.getcwd()
root_directory = f'{current_directory}/data/Vesuvius'
process_nrrd_files(root_directory)

Processing: /home/james/Documents/VS/3dUnet-tri-class/data/Vesuvius/train/original_labels/manual_2_iters_40_rot_1_densified_label.nrrdProcessing: /home/james/Documents/VS/3dUnet-tri-class/data/Vesuvius/train/original_labels/3606_4000_8706_xyz_256_res1_s4_label.nrrdProcessing: /home/james/Documents/VS/3dUnet-tri-class/data/Vesuvius/train/original_labels/3606_4000_8450_xyz_256_res1_s4_label.nrrdProcessing: /home/james/Documents/VS/3dUnet-tri-class/data/Vesuvius/train/original_labels/manual_1_iters_20_rot_2_densified_label.nrrdProcessing: /home/james/Documents/VS/3dUnet-tri-class/data/Vesuvius/train/original_labels/manual_2_iters_20_rot_0_densified_label.nrrdProcessing: /home/james/Documents/VS/3dUnet-tri-class/data/Vesuvius/train/original_labels/manual_2_iters_40_rot_2_densified_label.nrrdProcessing: /home/james/Documents/VS/3dUnet-tri-class/data/Vesuvius/train/original_labels/3606_4256_8706_xyz_256_res1_s4_label.nrrdProcessing: /home/james/Documents/VS/3dUnet-tri-class/data/Vesuvius/tra

In [12]:
# plot the tri-class segmentation label
init_slice = 0
current_directory = os.getcwd()
data_split = 'train'
data_type = 'weight'
filename = 'manual_2_iters_76_rot_0_densified'
label_type = 'weights'
# filename = 'tri_class_manual_1_iters_76_rot_0_densified_label.nrrd'
pred = nrrd.read(f'{current_directory}/data/Vesuvius/{data_split}/{data_type}/{label_type}_{filename}_label.nrrd')[0]
print(np.unique(pred))
# pred = nrrd.read(f'{os.getcwd()}/data/Vesuvius/val/raw/manual_1_raw.nrrd')[0]
#show final clipped instance segmentation
def plot_slice(slice_index, axis=0):
    plt.figure(figsize=(8, 6))
    if axis == 1:
        plt.imshow(pred[:,slice_index,:])
    elif axis == 2:
        plt.imshow(pred[:,:,slice_index])
    else:
        plt.imshow(pred[slice_index,:,:])
    plt.colorbar()
    plt.title(f'Slice {slice_index}')
    plt.show()

interact(plot_slice, slice_index=IntSlider(min=0, max=pred.shape[0]-1, step=1, value=init_slice), axis=IntSlider(min=0, max=2, step=1, value=0))

FileNotFoundError: [Errno 2] No such file or directory: '/home/james/Documents/VS/3dUnet-tri-class/data/Vesuvius/train/weight/weights_manual_2_iters_76_rot_0_densified_label_label.nrrd'

In [3]:
#plot the tri class label on raw data
current_directory = os.getcwd()
# filename = 'manual_1_iters_76_rot_1_densified'
pred = nrrd.read(f'{current_directory}/data/Vesuvius/{data_split}/label/tri_class_{filename}_label.nrrd')[0]
raw_data = nrrd.read(f'{current_directory}/data/Vesuvius/{data_split}/raw/{filename}_raw.nrrd')[0]

if raw_data.dtype != np.int16 and raw_data.dtype != np.int32:
        normalized_arr = (raw_data - raw_data.min()) / (raw_data.max() - raw_data.min())
        raw_data = (normalized_arr * np.iinfo(np.int16).max).astype(np.int16)

# pred = nrrd.read(f'{os.getcwd()}/data/Vesuvius/val/raw/manual_1_raw.nrrd')[0]
#show final clipped instance segmentation
def plot_slice(slice_index, axis=0):
    plt.figure(figsize=(8, 6))
    if axis == 1:
        plt.imshow(mark_boundaries_color(raw_data[:,slice_index,:], pred[:,slice_index,:]))
    elif axis == 2:
        plt.imshow(mark_boundaries_color(raw_data[:,:,slice_index], pred[:,:,slice_index]))
    else:
        plt.imshow(mark_boundaries_color(raw_data[slice_index,:,:], pred[slice_index,:,:]))
    plt.colorbar()
    plt.title(f'Slice {slice_index}')
    plt.show()

interact(plot_slice, slice_index=IntSlider(min=0, max=pred.shape[0]-1, step=1, value=init_slice), axis=IntSlider(min=0, max=2, step=1, value=0))

interactive(children=(IntSlider(value=0, description='slice_index', max=255), IntSlider(value=0, description='…

<function __main__.plot_slice(slice_index, axis=0)>

In [4]:
data_type='label'
pred = nrrd.read(f'{current_directory}/data/Vesuvius/{data_split}/{data_type}/tri_class_{filename}_label.nrrd')[0]
labeled_arr = label_foreground_structures(pred)
img_path = f'{current_directory}/data/Vesuvius/{data_split}/raw/{filename}_raw.nrrd'
image, _ = nrrd.read(img_path)
if image.dtype != np.int16 and image.dtype != np.int32:
        normalized_arr = (image - image.min()) / (image.max() - image.min())
        image = (normalized_arr * np.iinfo(np.int16).max).astype(np.int16)
gt_label = nrrd.read(f'{current_directory}/data/Vesuvius/{data_split}/original_labels/{filename}_label.nrrd')[0]
print(np.unique(gt_label))

Number of connected foreground structures before filtering: 16
Number of connected foreground structures after filtering: 16
[ 0  1  2  4  5  6  7  8  9 10 11 12 13 14]


In [5]:
#plot tri-class recovered instance labels
def plot_slice(slice_index, axis=0):
    plt.figure(figsize=(8, 6))
    if axis == 1:
        plt.imshow(mark_boundaries_color(image[:,slice_index,:], labeled_arr[:,slice_index,:]))
    elif axis == 2:
        plt.imshow(mark_boundaries_color(image[:,:,slice_index], labeled_arr[:,:,slice_index]))
    else:
        plt.imshow(mark_boundaries_color(image[slice_index,:,:], labeled_arr[slice_index,:,:]))
    plt.colorbar()
    plt.title(f'Slice {slice_index}')
    plt.show()

interact(plot_slice, slice_index=IntSlider(min=0, max=pred.shape[0]-1, step=1, value=init_slice), axis=IntSlider(min=0, max=2, step=1, value=0))

interactive(children=(IntSlider(value=0, description='slice_index', max=255), IntSlider(value=0, description='…

<function __main__.plot_slice(slice_index, axis=0)>

In [6]:
#plot gt labels
pred2 = nrrd.read(f'{current_directory}/data/Vesuvius/{data_split}/original_labels/{filename}_label.nrrd')[0]
#show final clipped instance segmentation
def plot_slice(slice_index, axis=0):
    plt.figure(figsize=(8, 6))
    if axis == 1:
        plt.imshow(pred2[:,slice_index,:])
    elif axis == 2:
        plt.imshow(pred2[:,:,slice_index])
    else:
        plt.imshow(pred2[slice_index,:,:])
    plt.colorbar()
    plt.title(f'Slice {slice_index}')
    plt.show()

interact(plot_slice, slice_index=IntSlider(min=0, max=pred2.shape[0]-1, step=1, value=init_slice), axis=IntSlider(min=0, max=2, step=1, value=0))

interactive(children=(IntSlider(value=0, description='slice_index', max=255), IntSlider(value=0, description='…

<function __main__.plot_slice(slice_index, axis=0)>

In [7]:
import os
import h5py
import nrrd
import numpy as np

def create_hdf5_files(raw_dir, label_dir, output_dir, weight_dir=None):
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    for raw_filename in os.listdir(raw_dir):
        if raw_filename.endswith('.nrrd'):
            # Construct full file paths for raw, weights and label
            raw_path = os.path.join(raw_dir, raw_filename)
            base_name = '_'.join(raw_filename.split('_')[:-1])
            weight_filename = "weights_"+base_name+'_label.nrrd'
            label_filename = "tri_class_"+base_name+'_label.nrrd'
            label_path = os.path.join(label_dir, label_filename)

            print(f"Processing: {raw_filename} and {label_filename}")
            
            if not os.path.exists(label_path):
                print(f"Label file not found for {label_filename}")
                label_filename = f"tri_class_{raw_filename}"
                print(f"trying {label_filename}")
                label_path = os.path.join(label_dir, label_filename)
            if not os.path.exists(label_path):
                print(f"Label file not found for {raw_filename}, skipping.")
                continue
            
            # Read raw and label data
            raw_data, raw_header = nrrd.read(raw_path)
            if raw_data.dtype != np.int16 and raw_data.dtype != np.int32:
                normalized_arr = (raw_data - raw_data.min()) / (raw_data.max() - raw_data.min())
                raw_data = (normalized_arr * np.iinfo(np.int16).max).astype(np.int16)
            label_data, label_header = nrrd.read(label_path)
            
            # Optionally read weight data
            if weight_dir:
                weight_path = os.path.join(weight_dir, weight_filename)
                if os.path.exists(weight_path):
                    weight_data, weight_header = nrrd.read(weight_path)
            
            raw_dtype = np.int16
            # Convert to numpy arrays
            raw_data = np.asarray(raw_data, dtype=raw_dtype)
            label_data = np.asarray(label_data, dtype=np.uint8)
            if weight_data is not None:
                weight_data = np.asarray(weight_data, dtype=raw_dtype)
            
            # Construct output file path
            output_filename = base_name + '.h5'
            output_path = os.path.join(output_dir, output_filename)
            
            # Create HDF5 file
            with h5py.File(output_path, 'w') as hdf5_file:
                hdf5_file.create_dataset('raw', data=raw_data, dtype='int16')
                hdf5_file.create_dataset('label', data=label_data, dtype='uint8')
                if weight_data is not None:
                    hdf5_file.create_dataset('weight', data=weight_data, dtype='uint8')
            
            print(f"Processed and saved {raw_filename} and {label_filename} to {output_path}")

In [ ]:
current_directory = os.getcwd()
sub_dir = 'test'
raw_directory = f'{current_directory}/data/Vesuvius/{sub_dir}/raw'
label_directory = f'{current_directory}/data/Vesuvius/{sub_dir}/label'
output_hdf5_path = f'{current_directory}/data/Vesuvius/{sub_dir}/dataset'
create_hdf5_files(raw_directory, label_directory, output_hdf5_path)

In [ ]:
sub_dir = 'val'
raw_directory = f'{current_directory}/data/Vesuvius/{sub_dir}/raw'
label_directory = f'{current_directory}/data/Vesuvius/{sub_dir}/label'
weight_directory = f'{current_directory}/data/Vesuvius/{sub_dir}/weight'
output_hdf5_path = f'{current_directory}/data/Vesuvius/{sub_dir}/dataset'
create_hdf5_files(raw_directory, label_directory, output_hdf5_path)

In [8]:
sub_dir = 'train'
raw_directory = f'{current_directory}/data/Vesuvius/{sub_dir}/raw'
label_directory = f'{current_directory}/data/Vesuvius/{sub_dir}/label'
weight_directory = f'{current_directory}/data/Vesuvius/{sub_dir}/weight'
output_hdf5_path = f'{current_directory}/data/Vesuvius/{sub_dir}/dataset'
create_hdf5_files(raw_directory, label_directory, output_hdf5_path, weight_directory)

Processing: manual_1_raw.nrrd and tri_class_manual_1_label.nrrd
Processed and saved manual_1_raw.nrrd and tri_class_manual_1_label.nrrd to /home/james/Documents/VS/3dUnet-tri-class/data/Vesuvius/train/dataset/manual_1.h5


In [ ]:
#visualise a slice from each training dataset
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt

def display_h5_slices(folder_path, slice_index=0):
    for file in os.listdir(folder_path):
        if file.endswith('.h5'):
            file_path = os.path.join(folder_path, file)
            with h5py.File(file_path, 'r') as h5_file:
                if 'raw' in h5_file and 'label' in h5_file:
                    raw_data = h5_file['raw']
                    label_data = h5_file['label']
                    
                    # Check if the slice_index is within bounds
                    if slice_index < raw_data.shape[0] and slice_index < label_data.shape[0]:
                        raw_slice = raw_data[slice_index, :, :]
                        print(raw_slice.dtype)
                        label_slice = label_data[slice_index, :, :]
                        
                        fig, axes = plt.subplots(1, 2, figsize=(10, 5))
                        axes[0].imshow(raw_slice, cmap='gray')
                        axes[0].set_title('Raw Data')
                        axes[1].imshow(mark_boundaries_color(raw_slice, label_slice))
                        axes[1].set_title('Label Data')
                        plt.suptitle(f"File: {file}, Slice: {slice_index}")
                        plt.show()
                    else:
                        print(f"Slice index {slice_index} out of bounds for file: {file}")

current_directory = os.getcwd()
h5_folder = f'{current_directory}/data/Vesuvius/train/dataset'
display_h5_slices(h5_folder)